<a href="https://colab.research.google.com/github/tperellom/Projectes/blob/develop/NN_con_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries and packages

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import joblib
import os
import tensorflow.keras.backend as K

In [ ]:
print(tf.__version__)

2.4.0


In [ ]:
from io import BytesIO
from PIL import Image
import base64
import io
import cv2

In [ ]:
!pip install flask-ngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!mkdir '/content/templates'

## Model

In [ ]:
model_file = "/content/drive/MyDrive/MSP/model.h5"

model = tf.keras.models.load_model(model_file)

In [ ]:
img_height, img_width = model.input.shape[1], model.input.shape[2]

In [ ]:
label_dict = {
    '0': 'Class A',
    '1': 'Class B',
    '2': 'Class C'
}

In [ ]:
def keras_preprocessing(img_path, img_height, img_width):
  img = tf.keras.preprocessing.image.load_img(img_path, target_size = (img_height, img_width))
  img = tf.keras.preprocessing.image.img_to_array(img) / 255.
  img = np.expand_dims(img, axis=0)
  
  return img

In [ ]:
def keras_raw_classifier(imagefile, img_height, img_width):
  input = keras_preprocessing(imagefile, img_height, img_width)
  output = model.predict(input)
  output_label = output.argmax()
 
  return output_label

In [ ]:
def preprocessing(imagefile, img_height, img_width):
  img = imagefile / 255. 
  img = np.expand_dims(img, axis=0)
  
  return img

In [ ]:
def classifier(imagefile, img_height, img_width):
  input = preprocessing(imagefile, img_height, img_width)
  output = model.predict(input)
  output_label = output.argmax()
 
  return output_label

##Browser API

In [ ]:
from flask import Flask, request, Response, jsonify, render_template, send_file
from flask_ngrok import run_with_ngrok

In [ ]:
html_code = '''
<!doctype html>
<html>
  <head>
    <title>Prueba</title>
    <style>
    	html * {
   			font-size: 1em !important;
   			font-family: Arial !important;
		}
	</style>
  </head>

  <body>
  <div>
      <h3>Reconocimiento de imágenes</h3>
      <p>Subir imagen</p>
      <form action="" method=post enctype=multipart/form-data>
        <input type=file name=file>
        <input type=submit>
      </form>
  </div>
  <div>
  	<h3>Resultado</h3>
    <td> <img src="data:image/png; base64, {{user_input}}"> </td>
    <td> <p><b>Clasificación:</b> {{result}}</p></td>
  </div>
  </body>
</html>
'''

In [ ]:
Html_file= open("/content/templates/index.html","w")
Html_file.write(html_code)
Html_file.close()

In [ ]:
# Initialize the Flask application
app = Flask(__name__)
run_with_ngrok(app)  

@app.route('/')
def home():
    return render_template('index.html')

# route http posts to this method
@app.route('/', methods=['POST'])
def main():
    # Load the input image
    img = request.files["file"].read()
    img = Image.open(BytesIO(img))
    img = img.convert('RGB')
    img = img.resize((img_height, img_width))
    
    # Classify the image by means of the neural network
    input = np.array(img)
    lab = classifier(input, img_height, img_width)
    res = label_dict[str(lab)]

    # Encode the image for HTML 
    img_html = cv2.cvtColor(input, cv2.COLOR_RGB2BGR)
    _, buffer = cv2.imencode('.png', img_html)
    img_dcd = base64.b64encode(buffer).decode('utf-8')

    return render_template('index.html', result=res, user_input=img_dcd)
    
# start flask app
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a0e4836f343a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Dec/2020 12:59:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2020 12:59:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Dec/2020 12:59:11] "POST / HTTP/1.1" 200 -


## Mailing service using smtplib

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from jinja2 import Template

In [ ]:
def image_classifier(img_file):

  res = keras_raw_classifier(img_file, img_height, img_width)

  processed_img = tf.keras.preprocessing.image.load_img(img_file, target_size=(img_height, img_width))

  # Encode the image for HTML 
  img_html = cv2.cvtColor(np.array(processed_img), cv2.COLOR_RGB2BGR)
  _, buffer = cv2.imencode('.png', img_html)
  img_dcd = base64.b64encode(buffer).decode('utf-8')

  return (res, img_dcd)

In [ ]:
html_message = '''
<!doctype html>
<html>
  <head>
    <title>Prueba</title>
    <style>
    	html * {
   			font-size: 1em !important;
   			font-family: Arial !important;
		}
	</style>
  </head>

  <body>
  <div>
  	<h3>Servicio de alerta</h3>
    <p>Esto es un mensaje automático generado por el sistema de vigilancia</p>
    <p>Por favor, compruebe la siguiente imagen:</p>
    <td> <img src="data:image/png; base64, {{user_input}}"> </td>
    <td> <p><b>Clasificación:</b> {{result}}</p></td>
  </div>
  </body>
</html>
'''

In [ ]:
img_file = 'Test1.jpg'
x, y = image_classifier(img_file)

# If the classification label is 1 or 2, send a message:
if x == 1 or x == 2:
  x_lab = label_dict[str(x)]

  # Mail instance:
  mail = MIMEMultipart("related")
  mail['Subject'] = 'Test Aviso'
  mail['From'] = "no-reply <7fdd917588-15db0d@inbox.mailtrap.io>"
  mail['To'] = "Test User <roxegid446@ofdyn.com>"

  # Jinja2 for html template
  main_template = Template(html_message)
  html = main_template.render(result=x_lab, user_input=y)
  msg_html = MIMEText(html, 'html')
  mail.attach(msg_html)

  # Send the mail
  with smtplib.SMTP("smtp.mailtrap.io", 2525) as server:
    server.login("6176c96091480a", "1107e733c3efa7")
    server.sendmail(mail['From'], mail['To'], mail.as_string().encode())
    server.quit()